In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/input_configuration.toml'))
summary_config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/summary_configuration.toml'))

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
output_path = os.path.join(summary_config['sc_run_path'], summary_config["output_folder"])
survey_path = os.path.join(summary_config['sc_run_path'], summary_config["survey_folder"])
output_dir = os.path.join(output_path,'RTP\person')

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

## Trips per Day by Resident

In [5]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(os.path.join(output_path,'agg\dash\person_trips.csv'))
person = pd.read_csv(os.path.join(output_path,'agg\dash\person_geog.csv'))

df_trip['is_rgc'] = 0
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person = person.copy()
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

In [6]:
# test = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parcel_2023_geography.csv')

In [7]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person['psexpfac'].sum()
    df.loc['Region', 'trexpfac'] = df_trip['trexpfac'].sum()
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    # df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    # df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        colname = purp + ' Trips'
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': colname}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region'] = df_trip[df_trip['trip_type'] == purp]['trexpfac'].sum()
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [8]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1

In [9]:
df = trips_per_day('is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
Not in RGC,0.5,3.2,3.7
In RGC,0.7,3.3,4.0
Region,0.5,3.2,3.7


In [10]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.4,3.4,3.9
Bellevue,0.7,3.4,4.1
Bothell Canyon Park,0.6,3.3,3.9
Bremerton,0.5,3.5,4.0
Burien,0.5,3.3,3.8
Everett,0.5,3.4,4.0
Federal Way,0.4,3.5,3.9
Greater Downtown Kirkland,0.6,3.4,3.9
Kent,0.4,3.4,3.9


In [11]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.2,3.7
Core,0.5,3.3,3.8
HCT,0.5,3.2,3.7
Metro,0.6,3.3,3.9
UU,0.4,3.2,3.6
Region,0.5,3.2,3.7


In [12]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1]['psexpfac'].sum()
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = pd.concat([trip_rate_df,_rate_df]) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']
df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_disability_reg,0.5,3.2,3.7
hh_elderly_reg,1.4,3.2,4.6
hh_english_reg,0.5,3.3,3.8
hh_poverty_50,0.4,3.2,3.6
hh_poverty_reg,0.5,3.2,3.7
hh_racial_50,0.5,3.3,3.8
hh_racial_reg,0.5,3.3,3.8
hh_youth_reg,0.5,3.2,3.7


In [13]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 0]['psexpfac'].sum()
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = pd.concat([trip_rate_df,_rate_df]) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']

df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df['Equity Group'] = ['NOT in '+i for i in df['Equity Group']]
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
NOT in hh_disability_reg,0.4,2.6,3.0
NOT in hh_elderly_reg,1.2,2.9,4.1
NOT in hh_english_reg,0.3,2.0,2.4
NOT in hh_poverty_50,0.0,0.1,0.1
NOT in hh_poverty_reg,0.3,2.1,2.4
NOT in hh_racial_50,0.2,1.5,1.7
NOT in hh_racial_reg,0.4,2.8,3.2
NOT in hh_youth_reg,0.5,3.2,3.7


## Miles Driven per Day by Resident

In [14]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(os.path.join(output_path,'agg\dash\person_vmt.csv'))
df_person = person.copy()

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [15]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,13.4
Kitsap,14.0
Outside Region,16.2
Pierce,15.1
Snohomish,16.3
Region,14.4


In [16]:
df = vmt_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Miles per Person
Not in RGC,14.9
In RGC,6.2
Region,14.4


In [17]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,11.3
Bellevue,7.0
Bothell Canyon Park,15.5
Bremerton,6.8
Burien,12.6
Everett,9.4
Federal Way,11.0
Greater Downtown Kirkland,11.7
Kent,10.0


In [18]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,17.6
Core,13.9
HCT,14.7
Metro,9.9
UU,16.0
Region,13.2



Average Miles per Person (by residence inside or outside equity geography)

In [19]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type

            df = vmt_per_person(df_vmt, df_person, col).reset_index().T
            df.columns = ['Outside Equity Geography', 'Inside Equity Geography',  'Region']
            df['source'] = col
            df = df.loc[["Average Miles per Person"]][['source','Inside Equity Geography',  'Outside Equity Geography']].reset_index(drop=True)
            
            result_df = pd.concat([result_df,df])

result_df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
result_df

,source,Inside Equity Geography,Outside Equity Geography
0,hh_youth_reg,15.8,12.9
0,hh_elderly_reg,15.2,13.6
0,hh_english_reg,13.7,14.8
0,hh_racial_reg,13.0,15.5
0,hh_racial_50,12.9,15.0
0,hh_poverty_reg,13.0,15.3
0,hh_poverty_50,9.9,14.4
0,hh_disability_reg,14.1,14.5


## Miles Walking and Biking per Day by Resident

In [20]:
df_vht = pd.read_csv(os.path.join(output_path,'agg\dash\person_vht.csv'))
df_hh = pd.read_csv(os.path.join(output_path,'agg\dash\hh_geog.csv'))



# def walk_bike_per_person_time(df_vmt, df_person, geog):
    
#     results_df = pd.DataFrame()
#     for mode in ['Walk','Bike']:
        
#         _df_vmt = df_vmt[df_vmt['mode'] == mode]
#         _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
#         _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

#         df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
#         df.loc['Region',:] = df.sum(axis=0)
#         df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
#         results_df[mode] = df['Average Miles per Person']
    
#     _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
#     _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
#     _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

#     df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
#     df.loc['Region',:] = df.sum(axis=0)
#     df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
#     results_df['Walk and Bike'] = df['Average Miles per Person']
    
#     return results_df

# df = walk_bike_per_person_time(df_vht, df_hh, 'hh_county')
# # df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
# df

In [21]:
# df = pd.read_csv(r'..\..\..\outputs\agg\dash\trip_distance.csv')
# _df = df[df['mode'] == 'Bike']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [22]:
# _df = df[df['mode'] == 'Walk']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [23]:
df_vmt = pd.read_csv(os.path.join(output_path,'agg\dash\person_vmt.csv'))
df_person = person.copy()

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1


def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

In [24]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.6,0.2,0.8
Kitsap,0.5,0.1,0.7
Pierce,0.5,0.2,0.6
Snohomish,0.5,0.2,0.6
Region,0.5,0.2,0.7


In [25]:
df = walk_bike_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'Inside RGC', 'Region']
df

,Walk,Bike,Walk and Bike
Not in RGC,0.5,0.2,0.7
Inside RGC,1.0,0.3,1.3
Region,0.5,0.2,0.7


In [26]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.7,0.3,1.0
Bellevue,0.9,0.3,1.1
Bothell Canyon Park,0.6,0.3,0.8
Bremerton,0.9,0.2,1.1
Burien,0.6,0.3,0.8
Everett,0.7,0.2,0.9
Federal Way,0.7,0.3,1.0
Greater Downtown Kirkland,0.6,0.2,0.9
Kent,0.6,0.3,1.0


In [27]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.4,0.1,0.6
Core,0.5,0.2,0.7
HCT,0.5,0.2,0.7
Metro,0.7,0.2,1.0
UU,0.4,0.2,0.6
Region,0.6,0.2,0.8


In [28]:
result_df_in = pd.DataFrame()
result_df_out = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)

            df_in = df.loc[[1]].reset_index(drop=True)
            df_in['source'] = col
            df_out = df.loc[[0]].reset_index(drop=True)
            df_out['source'] = "Not " + col
            
            result_df_in = pd.concat([result_df_in,df_in[['source','Walk','Bike','Walk and Bike']]])
            result_df_out = pd.concat([result_df_out,df_out[['source','Walk','Bike','Walk and Bike']]])
result_df_in

,source,Walk,Bike,Walk and Bike
0,hh_youth_reg,0.5,0.2,0.6
0,hh_elderly_reg,0.5,0.2,0.7
0,hh_english_reg,0.5,0.2,0.7
0,hh_racial_reg,0.6,0.2,0.8
0,hh_racial_50,0.6,0.2,0.8
0,hh_poverty_reg,0.6,0.2,0.8
0,hh_poverty_50,0.7,0.3,1.0
0,hh_disability_reg,0.5,0.2,0.7


In [29]:
result_df_out

,source,Walk,Bike,Walk and Bike
0,Not hh_youth_reg,0.6,0.2,0.8
0,Not hh_elderly_reg,0.6,0.2,0.8
0,Not hh_english_reg,0.5,0.2,0.7
0,Not hh_racial_reg,0.5,0.2,0.7
0,Not hh_racial_50,0.5,0.2,0.7
0,Not hh_poverty_reg,0.5,0.2,0.7
0,Not hh_poverty_50,0.5,0.2,0.7
0,Not hh_disability_reg,0.5,0.2,0.7


In [30]:
result_df = pd.concat([result_df_in,result_df_out])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1